In [1]:
## first step in creating new BkAvgAst... file
import numpy as np
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import os
os.chdir('C:/BankRisk')

df2 = pd.read_csv("df2_2001_2015.csv")
df2['AsOf'] = pd.to_datetime(df2['AsOf'])

In [2]:
fcol = df2.columns[5:]  ## get listing of numeric fields, then...
df2[fcol] = df2[fcol].astype(float)  ### CRITICAL to convert numeric data to FLOAT!!! (since some are ints)

In [4]:
df3 = df2.sort_index(by=['IDRSSD','AsOf'], ascending=True).reset_index(drop=True)
df3.head()

Unnamed: 0  IDRSSD       AsOf                    Name State     Cert  \
0           0      37 2001-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   
1        8857      37 2001-06-30  BANK OF HANCOCK COUNTY    GA  10057.0   
2       17647      37 2001-09-30  BANK OF HANCOCK COUNTY    GA  10057.0   
3       26405      37 2001-12-31  BANK OF HANCOCK COUNTY    GA  10057.0   
4       35094      37 2002-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   

   eTotAst   eTotLn   eALLL  eOREO   ...     ncoFIfor  ncoAg  ncoCIdom  \
0  65486.0  40507.0   979.0  200.0   ...          0.0    0.0       0.0   
1  65427.0  42153.0  1049.0    0.0   ...          0.0    0.0       0.0   
2  65575.0  44359.0  1070.0   40.0   ...          0.0    0.0       0.0   
3  66624.0  41514.0   987.0   20.0   ...          0.0    0.0       5.0   
4  68766.0  41539.0  1048.0   20.0   ...          0.0    0.0       0.0   

   ncoCIfor  ncoCard  ncoAuto  ncoOthCsmr  ncoForGovt  ncoOthLn  ncoLeas  
0       0.0      0.0      0.0        -1.0         0.0       0.0      0.0  
1       0.0      0.0      0.0       -23.0         0.0       0.0      0.0  
2       0.0      0.0      0.0         2.0         0.0       0.0      0.0  
3       0.0      0.0      0.0       104.0         0.0       0.0      0.0  
4       0.0      0.0      0.0       -13.0         0.0       0.0      0.0  

[5 rows x 193 columns]

In [5]:
len(df3)

456899

In [7]:
'eDDep' in df3.columns

True

In [8]:
# get mean eTotAst by IDRSSD:
df4 = df3[['IDRSSD','AsOf','Name','State','Cert','eTotAst','aTotAst','eTotLn','eDDep','eFDep']]
df4['eTotDep'] = df4['eDDep'] + df4['eFDep']
df5 = df4[df4['eTotAst']>0].groupby('IDRSSD')

In [9]:
df6x = pd.DataFrame(df5['AsOf'].max()).reset_index()
df6n = pd.DataFrame(df5['AsOf'].min()).reset_index()

In [10]:
df6n.tail()

IDRSSD       AsOf
10757  4368351 2012-03-31
10758  4536084 2013-12-31
10759  4569167 2013-12-31
10760  4589857 2013-12-31
10761  4845861 2015-09-30

In [11]:
df4.tail()

IDRSSD       AsOf                            Name State     Cert  \
456894  4589857 2014-03-31  INVESCO NATIONAL TRUST COMPANY    GA  59078.0   
456895  4589857 2014-06-30  INVESCO NATIONAL TRUST COMPANY    GA  59078.0   
456896  4589857 2014-09-30  INVESCO NATIONAL TRUST COMPANY    GA  59078.0   
456897  4845861 2015-09-30                    PRIMARY BANK    NH  59086.0   
456898  4845861 2015-12-31                    PRIMARY BANK    NH  59086.0   

        eTotAst   aTotAst  eTotLn   eDDep  eFDep  eTotDep  
456894  41899.0  221920.0     0.0     0.0    0.0      0.0  
456895  40112.0   39954.0     0.0     0.0    0.0      0.0  
456896  39151.0   39117.0     0.0     0.0    0.0      0.0  
456897  33811.0   21767.0  2827.0  6226.0    0.0   6226.0  
456898  36441.0   35000.0  7683.0  8286.0    0.0   8286.0

In [12]:
df7 = df6x.merge(df4[['IDRSSD','AsOf','Name','State','Cert']],how='inner', left_on=['IDRSSD','AsOf'], right_on=['IDRSSD','AsOf'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert']
df7.head()

IDRSSD      xAsOf                             xName xState    xCert
0      37 2015-12-31            BANK OF HANCOCK COUNTY     GA  10057.0
1     242 2015-12-31  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0
2     279 2015-12-31       MINEOLA COMMUNITY BANK, SSB     TX  28868.0
3     354 2015-12-31                  BISON STATE BANK     KS  14083.0
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0

In [13]:
df7 = df7.merge(df6n[['IDRSSD','AsOf']],how='inner', left_on=['IDRSSD'], right_on=['IDRSSD'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert', 'nAsOf']
df7.tail()

IDRSSD      xAsOf                                   xName xState  \
10757  4368351 2015-12-31  KEY NATIONAL TRUST COMPANY OF DELAWARE     DE   
10758  4536084 2015-12-31                    BANK OF BIRD-IN-HAND     PA   
10759  4569167 2015-03-31                     NEW TRADITIONS BANK     FL   
10760  4589857 2014-09-30          INVESCO NATIONAL TRUST COMPANY     GA   
10761  4845861 2015-12-31                            PRIMARY BANK     NH   

         xCert      nAsOf  
10757  59069.0 2012-03-31  
10758  59074.0 2013-12-31  
10759  58822.0 2013-12-31  
10760  59078.0 2013-12-31  
10761  59086.0 2015-09-30

In [14]:
df5m = df5.mean()
df5m = df5m.reset_index()
df5m.drop('Cert', inplace=True, axis=1)
df8 = df7.merge(df5m, how='inner', left_on='IDRSSD', right_on='IDRSSD')

In [15]:
df8_orig = df8.copy()
len(df8)

10762

In [18]:
df8.head()

IDRSSD      xAsOf                             xName xState    xCert  \
0      37 2015-12-31            BANK OF HANCOCK COUNTY     GA  10057.0   
1     242 2015-12-31  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0   
2     279 2015-12-31       MINEOLA COMMUNITY BANK, SSB     TX  28868.0   
3     354 2015-12-31                  BISON STATE BANK     KS  14083.0   
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0   

       nAsOf        eTotAst        aTotAst         eTotLn          eDDep  \
0 2001-03-31   80639.433333   80248.116667   41709.550000   64532.400000   
1 2001-03-31   32453.750000   32790.616667   20858.883333   24818.950000   
2 2001-03-31  143088.866667  137887.750000   64189.866667   96462.450000   
3 2001-03-31    8947.100000    9022.300000    4044.616667    7782.666667   
4 2001-03-31  166510.615385  165717.384615  118665.038462  144540.423077   

   eFDep        eTotDep  
0    0.0   64532.400000  
1    0.0   24818.950000  
2    0.0   96462.450000  
3    0.0    7782.666667  
4    0.0  144540.423077

In [19]:
df8.columns = ['IDRSSD','xAsOf','xName','xState','xCert','nAsOf','aeTotAst','aaTotAst','aeTotLn','aeDDep','aeFDep','aeTotDep']
df8.tail()

IDRSSD      xAsOf                                   xName xState  \
10757  4368351 2015-12-31  KEY NATIONAL TRUST COMPANY OF DELAWARE     DE   
10758  4536084 2015-12-31                    BANK OF BIRD-IN-HAND     PA   
10759  4569167 2015-03-31                     NEW TRADITIONS BANK     FL   
10760  4589857 2014-09-30          INVESCO NATIONAL TRUST COMPANY     GA   
10761  4845861 2015-12-31                            PRIMARY BANK     NH   

         xCert      nAsOf       aeTotAst       aaTotAst        aeTotLn  \
10757  59069.0 2012-03-31    3719.750000    3705.375000       0.000000   
10758  59074.0 2013-12-31   75839.000000   67046.777778   60345.333333   
10759  58822.0 2013-12-31  518790.666667  518238.166667  336623.666667   
10760  59078.0 2013-12-31   97796.000000   75247.750000       0.000000   
10761  59086.0 2015-09-30   35126.000000   28383.500000    5255.000000   

              aeDDep  aeFDep       aeTotDep  
10757       0.000000     0.0       0.000000  
10758   57856.777778     0.0   57856.777778  
10759  462212.333333     0.0  462212.333333  
10760       0.000000     0.0       0.000000  
10761    7256.000000     0.0    7256.000000

In [20]:
ff = pd.read_csv("CBfails_det201604.csv")
ff2 = ff[['State', 'Year', 'Institution Name ','Cert ', 'Effective Date ']]
ff2.columns = ['State', 'Year', 'Name', 'Cert', 'EffDate']
ff2['FailDate'] = pd.to_datetime(ff2['EffDate'])
ff2.dropna(subset=['Cert'],inplace=True)
ff2.tail()

State  Year                             Name     Cert    EffDate  \
2123    MO  1970  FIRST STATE BANK OF BONNE TERRE  12227.0  8/24/1970   
2124    NJ  1970          EATONTOWN NATIONAL BANK  19276.0   8/7/1970   
2125    KY  1970       FARMERS BANK OF PETERSBURG  13164.0  6/25/1970   
2126    MI  1970   THE PEOPLES STATE SAVINGS BANK  14987.0  4/18/1970   
2127    IA  1970       STATE BANK OF PRAIRIE CITY  14571.0  2/22/1970   

       FailDate  
2123 1970-08-24  
2124 1970-08-07  
2125 1970-06-25  
2126 1970-04-18  
2127 1970-02-22

In [21]:
dff2 = ff2[ff2.duplicated(['Cert'])][['Name','Cert']]  ## find duplicate Certs
len(dff2)

28

In [22]:
ff2lst = list(dff2['Cert'])
len(ff2lst)                      ## list of dup Certs

28

In [23]:
ff2a = ff2[ff2.Cert.isin(ff2lst)]
ff2a.sort_index(by=['Cert','FailDate'],inplace=True)  ## sorted list of dup Certs
ff2a.head()

State  Year                              Name    Cert     EffDate  \
1307    TX  1988        FIRST CITY, TEXAS - DALLAS  1722.0   4/20/1988   
571     TX  1992    FIRST CITY, TEXAS - DALLAS, NA  1722.0  10/30/1992   
1308    TX  1988      FIRST CITY, TEXAS - BEAUMONT  3093.0   4/20/1988   
572     TX  1992  FIRST CITY, TEXAS - BEAUMONT, NA  3093.0  10/30/1992   
1306    TX  1988         FIRST CITY, TEXAS - BRYAN  3120.0   4/20/1988   

       FailDate  
1307 1988-04-20  
571  1992-10-30  
1308 1988-04-20  
572  1992-10-30  
1306 1988-04-20

In [24]:
## sections to dedup Cert field:

In [25]:
ff2x = ff2[['Cert','FailDate']].groupby('Cert').max()
ff2x.reset_index(drop=False,inplace=True)
ff2x['mx'] = 'x'
ff2x.tail()

Cert   FailDate mx
2090  58539.0 2011-04-29  x
2091  58701.0 2013-05-10  x
2092  90027.0 1988-04-15  x
2093  90047.0 1988-12-16  x
2094  91189.0 1998-07-23  x

In [26]:
ff3 = ff2.merge(ff2x,how='left',left_on=['Cert','FailDate'],right_on=['Cert','FailDate'])
ff3.tail()

State  Year                             Name     Cert    EffDate  \
2118    MO  1970  FIRST STATE BANK OF BONNE TERRE  12227.0  8/24/1970   
2119    NJ  1970          EATONTOWN NATIONAL BANK  19276.0   8/7/1970   
2120    KY  1970       FARMERS BANK OF PETERSBURG  13164.0  6/25/1970   
2121    MI  1970   THE PEOPLES STATE SAVINGS BANK  14987.0  4/18/1970   
2122    IA  1970       STATE BANK OF PRAIRIE CITY  14571.0  2/22/1970   

       FailDate mx  
2118 1970-08-24  x  
2119 1970-08-07  x  
2120 1970-06-25  x  
2121 1970-04-18  x  
2122 1970-02-22  x

In [27]:
ff4=ff3.dropna()
len(ff3), len(ff4)

(2123, 2095)

In [28]:
df8 = df8.merge(ff4[['Cert','FailDate']], how='left', left_on='xCert', right_on='Cert')
df8 = df8.drop('Cert', axis=1)
len(df8)

10762

In [29]:
df8.sort_index(by='aeTotAst',ascending=False, inplace=True)
df8.reset_index(drop=True, inplace=True)  # renumber rows based on largest to smallest banks
df8.reset_index(drop=False, inplace=True)  # move row numbers into another column called 'index'
df8.rename(columns={'index':'Rank'},inplace=True)  # rename 'index' column to 'Rank'
df8['Rank'] = df8['Rank'] + 1  # add 1 so that largest bank has rank 1 rather than rank 0
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2015-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2015-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2015-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2015-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.351213e+09  1.325373e+09  4.608670e+08  5.710782e+08   
1 2001-03-31  1.179955e+09  1.187259e+09  6.161152e+08  7.159990e+08   
2 2001-03-31  9.991729e+08  9.956328e+08  4.859796e+08  2.470630e+08   
3 2001-03-31  7.287602e+08  7.165592e+08  4.586744e+08  4.985467e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate  
0  2.527046e+08  8.237827e+08        NaT  
1  1.100090e+08  8.260080e+08 2009-01-16  
2  4.301490e+08  6.772120e+08 2008-11-23  
3  5.936475e+07  5.579114e+08        NaT  
4  2.469325e+07  2.951088e+08        NaT

In [30]:
df8['pr_eTA'] = df8['aeTotAst']*100/df8['aeTotAst'].sum()
df8['pr_eTAcs'] = df8['pr_eTA'].cumsum()
df8['pr_eTL'] = df8['aeTotLn']*100/df8['aeTotLn'].sum()
df8['pr_eTLcs'] = df8['pr_eTL'].cumsum()
df8['pr_eDD'] = df8['aeDDep']*100/df8['aeTotLn'].sum()
df8['pr_eDDcs'] = df8['pr_eDD'].cumsum()
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2015-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2015-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2015-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2015-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.351213e+09  1.325373e+09  4.608670e+08  5.710782e+08   
1 2001-03-31  1.179955e+09  1.187259e+09  6.161152e+08  7.159990e+08   
2 2001-03-31  9.991729e+08  9.956328e+08  4.859796e+08  2.470630e+08   
3 2001-03-31  7.287602e+08  7.165592e+08  4.586744e+08  4.985467e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.527046e+08  8.237827e+08        NaT  9.127740   9.127740  5.470572   
1  1.100090e+08  8.260080e+08 2009-01-16  7.970853  17.098593  7.313395   
2  4.301490e+08  6.772120e+08 2008-11-23  6.749630  23.848224  5.768663   
3  5.936475e+07  5.579114e+08        NaT  4.922934  28.771157  5.444545   
4  2.469325e+07  2.951088e+08        NaT  2.983418  31.754575  3.022585   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.470572  6.778798   6.778798  
1  12.783967  8.499033  15.277831  
2  18.552629  2.932681  18.210512  
3  23.997174  5.917837  24.128348  
4  27.019759  3.209880  27.338229

In [31]:
df8.to_csv("BkAvgAstPR_FailDate.csv")

In [32]:
df8_fail = df8[df8['FailDate'].map(lambda x: x.year) > 2000]   #cbt[cbt['AsOf'].map(lambda x: x.month) == 12] 
len(df8_fail)

478

In [33]:
df8_fail.to_csv("FailedBkList_AvgAst.csv")